# IE6511 Homework 8
Done by: Aloisius Stephen and Yang Xiaozhou

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pySOT import *
from poap.controller import SerialController, ThreadController, BasicWorkerThread
%matplotlib inline

np.set_printoptions(precision=3)

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 12}

matplotlib.rc('font', **font)

## 1. Using PySOT with constraint handling on Keane’s Bump function (10 Points)

## 2.	(Computer Usage) Setting up SO-MI in PySOT (15 points)

## 3.	Short Answer Questions (no computer necessary)

### 4.	Explain what “restart” is for some surrogate global optimization methods.Explain  when and why LMSRBF will restart and answer the same question for DYCORS. Is there a difference between the two methods.  Explain the advantages and disadvantages of restarting.

### 5.	Assume you have a 15 dimensional decision vector $X_i^A$, where $X_i^A$, A = LMSRBF or DYCORS to indicate the algorithm. So $X_i^L$ refers to the value of the decision vector in the I-th iteration of the LMSRBF algorithm. Let $n_o$ be the number of evaluations done for the Latin Hypercube evaluation. So what can you say about the expected Euclidean distance from $X_i^L$ to $X_{i+1}^L$ versus the expected Euclidean distance $X_i^D$ to  $X_{i+1}^D$?